In [1]:
# Required libraries
import os
import tarfile
import json
from pathlib import Path
from radiant_mlhub.client import _download as download_file

import datetime
import rasterio
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit

os.environ['MLHUB_API_KEY'] = '3d4d443d2fa0498b2cc1fe278b4936dfbd5e117aa4fe54f193507a6b9e502640'

In [13]:
DOWNLOAD_S1 = False # If you set this to true then the Sentinel-1 data will be downloaded which is not needed in this notebook.

# Select which imagery bands you'd like to download here:
DOWNLOAD_BANDS = {
    'B01': False,
    'B02': False,
    'B03': False,
    'B04': False,
    'B05': True,
    'B06': False,
    'B07': True,
    'B08': False,
    'B8A': False,
    'B09': False,
    'B11': False,
    'B12': False,
    'CLM': True
}

# In this model we will only use Green, Red and NIR bands. You can select to download any number of bands. 
# Our choice relies on the fact that vegetation is most sensitive to these bands. 
# We also donwload the CLM or Cloud Mask layer to exclude cloudy data from the training phase. 
# You can also do a feature selection, and try different combination of bands to see which ones will result in a better accuracy.

In [14]:
%%time
FOLDER_BASE = 'ref_south_africa_crops_competition_v1'

def download_archive(archive_name):
    if os.path.exists(archive_name.replace('.tar.gz', '')):
        return
    
    print(f'Downloading {archive_name} ...')
    download_url = f'https://radiant-mlhub.s3.us-west-2.amazonaws.com/archives/{archive_name}'
    download_file(download_url, '.')
    print(f'Extracting {archive_name} ...')
    with tarfile.open(archive_name) as tfile:
        tfile.extractall()
    os.remove(archive_name)

for split in ['train', 'test']:
    # Download the labels
    labels_archive = f'{FOLDER_BASE}_{split}_labels.tar.gz'
    download_archive(labels_archive)
    
    # Download Sentinel-1 data
    if DOWNLOAD_S1:
        s1_archive = f'{FOLDER_BASE}_{split}_source_s1.tar.gz'
        download_archive(s1_archive)
        

    for band, download in DOWNLOAD_BANDS.items():
        if not download:
            continue
        s2_archive = f'{FOLDER_BASE}_{split}_source_s2_{band}.tar.gz'
        download_archive(s2_archive)
        
def resolve_path(base, path):
    return Path(os.path.join(base, path)).resolve()
        
def load_df(collection_id):
    split = collection_id.split('_')[-2]
    collection = json.load(open(f'{collection_id}/collection.json', 'r'))
    rows = []
    item_links = []
    for link in collection['links']:
        if link['rel'] != 'item':
            continue
        item_links.append(link['href'])
        
    for item_link in item_links:
        item_path = f'{collection_id}/{item_link}'
        current_path = os.path.dirname(item_path)
        item = json.load(open(item_path, 'r'))
        tile_id = item['id'].split('_')[-1]
        for asset_key, asset in item['assets'].items():
            rows.append([
                tile_id,
                None,
                None,
                asset_key,
                str(resolve_path(current_path, asset['href']))
            ])
            
        for link in item['links']:
            if link['rel'] != 'source':
                continue
            source_item_id = link['href'].split('/')[-2]
            
            if source_item_id.find('_s1_') > 0 and not DOWNLOAD_S1:
                continue
            elif source_item_id.find('_s1_') > 0:
                for band in ['VV', 'VH']:
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s1/{source_item_id}/{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's1',
                        band,
                        asset_path
                    ])
                
            if source_item_id.find('_s2_') > 0:
                for band, download in DOWNLOAD_BANDS.items():
                    if not download:
                        continue
                    
                    asset_path = Path(f'{FOLDER_BASE}_{split}_source_s2_{band}/{source_item_id}_{band}.tif').resolve()
                    date = '-'.join(source_item_id.split('_')[10:13])
                    rows.append([
                        tile_id,
                        f'{date}T00:00:00Z',
                        's2',
                        band,
                        asset_path
                    ])
            
    return pd.DataFrame(rows, columns=['tile_id', 'datetime', 'satellite_platform', 'asset', 'file_path'])

competition_train_df = load_df(f'{FOLDER_BASE}_train_labels')
competition_test_df = load_df(f'{FOLDER_BASE}_test_labels')

  0%|          | 0/4536.5 [00:00<?, ?M/s]

Extracting ref_south_africa_crops_competition_v1_train_source_s2_B05.tar.gz ...


  0%|          | 0/4702.1 [00:00<?, ?M/s]

Extracting ref_south_africa_crops_competition_v1_train_source_s2_B07.tar.gz ...


  0%|          | 0/1930.4 [00:00<?, ?M/s]

Extracting ref_south_africa_crops_competition_v1_test_source_s2_B05.tar.gz ...


  0%|          | 0/2002.0 [00:00<?, ?M/s]

Extracting ref_south_africa_crops_competition_v1_test_source_s2_B07.tar.gz ...
Wall time: 1h 38min 22s


In [15]:
competition_train_df

,tile_id,datetime,satellite_platform,asset,file_path
0,2587,None,None,documentation,E:\ML_HUB\ref_south_africa_crops_competition_v...
1,2587,None,None,field_ids,E:\ML_HUB\ref_south_africa_crops_competition_v...
2,2587,None,None,field_info_train,E:\ML_HUB\ref_south_africa_crops_competition_v...
3,2587,None,None,labels,E:\ML_HUB\ref_south_africa_crops_competition_v...
4,2587,None,None,raster_values,E:\ML_HUB\ref_south_africa_crops_competition_v...
...,...,...,...,...,...
446643,2198,2017-11-27T00:00:00Z,s2,B07,E:\ML_HUB\ref_south_africa_crops_competition_v...
446644,2198,2017-11-27T00:00:00Z,s2,CLM,E:\ML_HUB\ref_south_africa_crops_competition_v...
446645,2198,2017-11-30T00:00:00Z,s2,B05,E:\ML_HUB\ref_south_africa_crops_competition_v...
446646,2198,2017-11-30T00:00:00Z,s2,B07,E:\ML_HUB\ref_south_africa_crops_competition_v...


In [16]:
%%time
competition_train_df.to_csv('competition_train3.csv',index = False)
competition_test_df.to_csv('competition_test3.csv', index = False)

Wall time: 3.55 s


In [17]:
# load data 
competition_train_df = pd.read_csv('competition_train3.csv')
competition_test_df = pd.read_csv('competition_test3.csv')

In [18]:
# This DataFrame lists all types of assets including documentation of the data. 
# In the following, we will use the Sentinel-2 bands as well as labels. 
competition_train_df['asset'].unique()

array(['documentation', 'field_ids', 'field_info_train', 'labels',
       'raster_values', 'B05', 'B07', 'CLM'], dtype=object)

In [19]:
tile_ids_train = competition_train_df['tile_id'].unique()

In [6]:
# For simplicty of this baseline model, we will use only 5 images throughout the growing season
# You can choose to use all of them, select a few of them at specifc intervals, or 
# load as many as you want and interpolate between them to have a regular temporal frequency.

# Another assumption is that we are selecting the first 5 cloud free images. Ideally, you should
# select the images across the different tiles with the same temporal frequency. 
n_obs = 5

In [20]:
%%time
# Our goal is developing a pixel-based Random Forest model. So we will create an X variable
# that each row is a pixel and each column is one of the observations. 
# The other variables is y which has rows equal to the number of pixels. 
X = np.empty((0, 2 * n_obs),dtype=np.float16)
y = np.empty((0, 1),np.int32)
field_ids = np.empty((0, 1),np.int32)

for tile_id in tile_ids_train:
    tile_df = competition_train_df[competition_train_df['tile_id']==tile_id]

    label_src = rasterio.open(tile_df[tile_df['asset']=='labels']['file_path'].values[0])
    label_array = label_src.read(1)
    y = np.append(y, label_array.flatten())

    field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array = field_id_src.read(1)
    field_ids = np.append(field_ids, field_id_array.flatten())

    tile_date_times = tile_df[tile_df['satellite_platform']=='s2']['datetime'].unique()

    X_tile = np.empty((256 * 256, 0))
    n_X = 0
    for date_time in tile_date_times:
        # Here we retrieve the cloud band, and check if it's cloud free we will load the other bands
        # Otherwise we will pass on to the next observation
        
        clm_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='CLM')]['file_path'].values[0])
        clm_max = np.max(clm_src.read(1))
        # In the following we select images that the maximum cloud cover probability per pixel is 10% (10% * 255 = 25.5).
        if clm_max < 25:
            n_X+=1
            b5_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B05')]['file_path'].values[0])
            b5_array = np.expand_dims(b5_src.read(1).flatten(), axis=1)
            
            b7_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B07')]['file_path'].values[0])
            b7_array = np.expand_dims(b7_src.read(1).flatten(), axis=1)

#             b8a_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B8A')]['file_path'].values[0])
#             b8a_array = np.expand_dims(b8a_src.read(1).flatten(), axis=1)

#             b11_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B11')]['file_path'].values[0])
#             b11_array = np.expand_dims(b11_src.read(1).flatten(), axis=1)
            
#             b12_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B12')]['file_path'].values[0])
#             b12_array = np.expand_dims(b12_src.read(1).flatten(), axis=1)


            X_tile = np.append(X_tile, b5_array, axis = 1)
            X_tile = np.append(X_tile, b7_array, axis = 1)
#             X_tile = np.append(X_tile, b8a_array, axis = 1)
#             X_tile = np.append(X_tile, b11_array, axis = 1)
#             X_tile = np.append(X_tile, b12_array, axis = 1)
        if n_X == n_obs:
            break
        
    X = np.append(X, X_tile, axis=0)

Wall time: 3h 29min 7s


In [21]:
data = pd.DataFrame(X)
data['label'] = y.astype(int)
data['field_id'] = field_ids
data = data[data.label != 0] #this filters the pixels that don't have a label (or corresponding field ID)
data

,0,1,2,3,4,5,6,7,8,9,label,field_id
2048,47.0,55.0,47.0,54.0,40.0,48.0,44.0,54.0,34.0,46.0,2,3020
2304,47.0,55.0,47.0,54.0,40.0,48.0,44.0,54.0,34.0,46.0,2,3020
2560,50.0,56.0,50.0,57.0,40.0,50.0,46.0,53.0,33.0,48.0,2,3020
2561,50.0,56.0,50.0,57.0,40.0,50.0,46.0,53.0,33.0,48.0,2,3020
2816,50.0,56.0,50.0,57.0,40.0,50.0,46.0,53.0,33.0,48.0,2,3020
...,...,...,...,...,...,...,...,...,...,...,...,...
173670320,96.0,104.0,89.0,96.0,88.0,96.0,87.0,95.0,89.0,94.0,9,73261
173670321,96.0,104.0,89.0,96.0,88.0,96.0,87.0,95.0,89.0,94.0,9,73261
173670322,93.0,101.0,86.0,92.0,88.0,92.0,84.0,88.0,88.0,96.0,9,73261
173670323,93.0,101.0,86.0,92.0,88.0,92.0,84.0,88.0,88.0,96.0,9,73261


In [22]:
# Each field has several pixels in the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id
data_grouped = data.groupby('field_id').mean().reset_index()
data_grouped

,field_id,0,1,2,3,4,5,6,7,8,9,label
0,0,55.183333,65.183333,54.700000,64.500000,53.033333,63.033333,51.733333,61.550000,48.800000,59.066667,5.483333
1,1,40.062701,55.641479,37.771704,51.930868,36.138264,46.610932,37.426045,45.440514,25.808682,46.670418,4.000000
2,2,54.695652,68.304348,53.217391,66.217391,42.086957,52.565217,43.391304,52.304348,35.391304,46.652174,7.000000
3,3,73.721080,87.759640,43.464010,56.005141,45.895887,57.195373,45.915167,56.219794,47.620823,56.124679,6.000000
4,4,68.470199,77.788079,59.887417,69.609272,45.397351,54.609272,53.284768,61.894040,65.953642,75.284768,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
87109,122731,52.459559,74.691176,51.518382,70.702206,50.246324,66.871324,43.014706,60.415441,36.937500,52.577206,4.000000
87110,122732,68.741557,74.751836,68.118943,74.651982,71.079295,77.823789,69.433186,75.323054,68.798825,75.538913,5.000000
87111,122733,30.430769,49.415385,28.984615,50.000000,26.507692,50.815385,27.076923,52.753846,28.092308,55.692308,2.000000
87112,122735,41.486111,123.638889,38.388889,120.958333,42.597222,118.013889,40.736111,111.986111,39.111111,106.458333,3.000000


In [23]:
%%time
data_grouped.to_csv('X_competition_train3.csv', index = False)

Wall time: 1.65 s


## Competition Test Data
In this part we will load the competition test data (which does not have labels) and predict the crop class for each field

In [24]:
tile_ids_test = competition_test_df['tile_id'].unique()

In [25]:
%%time
X_competition_test = np.empty((0, 2 * n_obs),dtype=np.float16)
field_ids_test = np.empty((0, 1),np.int32)

for tile_id in tile_ids_test:
    tile_df = competition_test_df[competition_test_df['tile_id']==tile_id]
    
    field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array = field_id_src.read(1)
    field_ids_test = np.append(field_ids_test, field_id_array.flatten())
    
    tile_date_times = tile_df[tile_df['satellite_platform']=='s2']['datetime'].unique()
    
    X_tile = np.empty((256 * 256, 0))
    n_X = 0
    for date_time in tile_date_times:
        # Here we retrieve the cloud band, and check if it's cloud free we will load the other bands
        # Otherwise we will pass on to the next observation
        
        clm_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='CLM')]['file_path'].values[0])
        clm_max = np.max(clm_src.read(1))
        
        if clm_max < 25:
            n_X+=1
            b5_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B05')]['file_path'].values[0])
            b5_array = np.expand_dims(b5_src.read(1).flatten(), axis=1)
            
            b7_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B07')]['file_path'].values[0])
            b7_array = np.expand_dims(b7_src.read(1).flatten(), axis=1)

#             b8a_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B8A')]['file_path'].values[0])
#             b8a_array = np.expand_dims(b8a_src.read(1).flatten(), axis=1)

#             b11_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B11')]['file_path'].values[0])
#             b11_array = np.expand_dims(b11_src.read(1).flatten(), axis=1)
            
#             b12_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='B12')]['file_path'].values[0])
#             b12_array = np.expand_dims(b12_src.read(1).flatten(), axis=1)

            X_tile = np.append(X_tile, b5_array, axis = 1)
            X_tile = np.append(X_tile, b7_array, axis = 1)
#             X_tile = np.append(X_tile, b8a_array, axis = 1)
#             X_tile = np.append(X_tile, b11_array, axis = 1)
#             X_tile = np.append(X_tile, b12_array, axis = 1)
        if n_X == n_obs:
            break
        
    X_competition_test = np.append(X_competition_test, X_tile, axis=0)

Wall time: 50min 25s


In [28]:
data_test = pd.DataFrame(X_competition_test)
data_test['field_id'] = field_ids_test
data_test = data_test[data_test.field_id != 0]
data_test

,0,1,2,3,4,5,6,7,8,9,field_id
72,71.0,78.0,68.0,77.0,71.0,80.0,72.0,79.0,65.0,75.0,102896
73,71.0,78.0,68.0,77.0,71.0,80.0,72.0,79.0,65.0,75.0,102896
74,67.0,75.0,65.0,72.0,68.0,75.0,66.0,75.0,62.0,69.0,102896
75,67.0,75.0,65.0,72.0,68.0,75.0,66.0,75.0,62.0,69.0,102896
76,69.0,75.0,67.0,74.0,69.0,76.0,69.0,75.0,63.0,71.0,102896
...,...,...,...,...,...,...,...,...,...,...,...
74514427,57.0,67.0,59.0,67.0,58.0,68.0,53.0,65.0,53.0,67.0,61236
74514428,62.0,70.0,61.0,70.0,62.0,71.0,57.0,68.0,55.0,68.0,61236
74514429,62.0,70.0,61.0,70.0,62.0,71.0,57.0,68.0,55.0,68.0,61236
74514430,65.0,72.0,62.0,70.0,62.0,72.0,59.0,69.0,56.0,68.0,61236


In [29]:
# Each field has several pixels in the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id
data_grouped1 = data_test.groupby('field_id').mean().reset_index()
data_grouped1

,field_id,0,1,2,3,4,5,6,7,8,9
0,5,55.903766,60.793584,55.736402,61.231520,48.453278,52.408647,40.218968,43.118550,32.708508,39.764296
1,10,40.805113,49.444766,39.086348,47.848046,35.301013,45.460685,31.723107,54.253256,32.587072,77.402798
2,11,66.956481,73.834437,60.148534,66.623463,68.792810,76.393567,68.145695,75.706717,69.073794,76.399243
3,17,73.727632,81.652632,73.540789,80.830263,71.560526,79.607895,75.813158,83.422368,74.225000,81.865789
4,18,45.715789,50.821053,43.473684,49.400000,42.000000,47.242105,43.600000,48.894737,42.400000,48.400000
...,...,...,...,...,...,...,...,...,...,...,...
35290,122722,65.256104,71.477922,61.106753,67.583636,66.210390,73.042078,63.894545,70.376364,59.796364,65.667532
35291,122724,51.497275,65.983651,49.760218,63.152589,41.316076,50.704360,40.396458,49.722071,26.445504,38.035422
35292,122726,59.604468,64.808147,61.002628,66.932983,60.367937,66.339028,50.520368,56.795007,49.258870,54.775296
35293,122730,44.594142,53.481172,43.820084,53.125523,43.878661,53.133891,41.493724,50.347280,41.690377,49.815900


In [30]:
%%time
data_grouped1.to_csv('X_competition_test3.csv', index = False)

Wall time: 633 ms
